# Instala las librerías necesarias:

`pip install selenium beautifulsoup4`

In [1]:
!pip install selenium beautifulsoup4

     |████████████████████████████████| 911kB 12.1MB/s eta 0:00:01


# Instalar Google Chrome en Linux

In [2]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

--2019-11-29 11:05:32--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolviendo dl.google.com (dl.google.com)... 172.217.7.174, 2607:f8b0:4004:800::200e
Conectando con dl.google.com (dl.google.com)[172.217.7.174]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 62950548 (60M) [application/x-debian-package]
Guardando como: “google-chrome-stable_current_amd64.deb”

google-chrome-stabl 100%[===================>]  60,03M  1,44MB/s    en 41s     

2019-11-29 11:06:13 (1,46 MB/s) - “google-chrome-stable_current_amd64.deb” guardado [62950548/62950548]



In [4]:
!echo 1234|sudo -S dpkg -i google-chrome-stable_current_amd64.deb

[sudo] contraseña para ingenieria: Seleccionando el paquete google-chrome-stable previamente no seleccionado.
(Leyendo la base de datos ... 191811 ficheros o directorios instalados actualmente.)
Preparando para desempaquetar google-chrome-stable_current_amd64.deb ...
Desempaquetando google-chrome-stable (78.0.3904.108-1) ...
Configurando google-chrome-stable (78.0.3904.108-1) ...
update-alternatives: utilizando /usr/bin/google-chrome-stable para proveer /usr/bin/x-www-browser (x-www-browser) en modo automático
update-alternatives: utilizando /usr/bin/google-chrome-stable para proveer /usr/bin/gnome-www-browser (gnome-www-browser) en modo automático
update-alternatives: utilizando /usr/bin/google-chrome-stable para proveer /usr/bin/google-chrome (google-chrome) en modo automático
Procesando disparadores para gnome-menus (3.13.3-11ubuntu1.1) ...
Procesando disparadores para desktop-file-utils (0.23-1ubuntu3.18.04.2) ...
Procesando disparadores para mime-support (3.60ubuntu1) ...
Procesan

# Bajar ChromeDriver

[Acá](https://chromedriver.chromium.org/downloads) puedes encontrar las diferentes versiones de ChromeDriver. Chequea tu versión de google Chrome para bajar el driver correcto

In [6]:
!google-chrome --version

Google Chrome 78.0.3904.108 


In [7]:
!ls

1_Introducción_a_Python_y_Pandas.ipynb	Pipfile
california_housing_train.csv		Pipfile.lock
chromedriver				ppc_suba_comentarios.json
chromedriver_linux64.zip		README.md
crawl_all_restaurants.ipynb		restaurantes_links.p
google-chrome-stable_current_amd64.deb	selenium_crawler.ipynb
online_shoppers_intention.csv		spider.py


In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [20]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.get("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

In [21]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
s = soup.select("#reviewList > li")

In [15]:
info = []
for review in soup.select("#reviewList > li"):
    text = review.select("[itemprop='description']")[0].get_text()
    author = review.select("div.small")[0].get_text()
    previousOrders = review.select("div.small")[1]["title"].split()[0]
    previousReviews = review.select("div.small")[2]["title"].split()[0]
    date = review.select("[itemprop='datePublished']")[0].get_text()
    worstRating = review.select("[itemprop='worstRating']")[0]["content"]
    ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
    bestRating = review.select("[itemprop='bestRating']")[0]["content"]
    infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                     date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
    info.append(infoReview)

In [16]:
print(info)

[{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto', 'author': 'Raul fernando', 'previousOrders': '2', 'previousReviews': '0', 'date': '21 sep, 2019', 'worstRating': '1', 'ratingValue': '3.7', 'bestRating': '5'}, {'text': 'El pollo más rico que tiene colombia ❤?❤?', 'author': 'Andres  ramos', 'previousOrders': '25', 'previousReviews': '0', 'date': '25 jul, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'redimi y cupón de $7.000 y me cobraron la cuenta completa sin el descuento del cupon', 'author': 'Camilo espitia', 'previousOrders': '266', 'previousReviews': '0', 'date': '20 jun, 2019', 'worstRating': '1', 'ratingValue': '4.0', 'bestRating': '5'}, {'text': 'Excelente servicio, el domiciliario es muy respetuoso y amable', 'author': 'Laura sanchez', 'previousOrders': '274', 'previousReviews': '0', 'date': '09 jun, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'Gratamente Sorprendida, han Mejorado la calida

In [22]:
button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
print(button.get_attribute("data-page"))
button.click()

2


In [23]:
driver.quit()

# Todo funciona perfecto! :-D

Ahora volvamos esto una función para iterar sobre todas las páginas de los comentarios de nuestro restaurante de ejemplo

In [40]:
import time

In [47]:
def download_domicilios(website):
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(website)
    driver.implicitly_wait(100)
    info = []
    pages = []
    while True:
        try:
            html = driver.page_source ###
            soup = BeautifulSoup(html, 'html.parser')  ###
            button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
            numPage = button.get_attribute("data-page")
            print('Currently at page {0}'.format(numPage))
            if numPage == '10':
                driver.quit()
                raise ValueError("numPage {0} was already visited".format(numPage))
            else:
                pages.append(numPage)
            for review in soup.select("#reviewList > li"):
                text = review.select("[itemprop='description']")[0].get_text()
                author = review.select("div.small")[0].get_text()
                previousOrders = review.select("div.small")[1]["title"].split()[0]
                previousReviews = review.select("div.small")[2]["title"].split()[0]
                date = review.select("[itemprop='datePublished']")[0].get_text()
                worstRating = review.select("[itemprop='worstRating']")[0]["content"]
                ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
                bestRating = review.select("[itemprop='bestRating']")[0]["content"]
                infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                                 date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
                info.append(infoReview)
            time.sleep(3)
            button.click()
        except Exception as e:
            print(e)
            break
    return info

In [48]:
info = download_domicilios("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

Currently at page 2
Currently at page 3
Currently at page 4
Currently at page 5
Currently at page 6
Currently at page 7
Currently at page 8
Currently at page 9
Currently at page 10
numPage 10 was already visited


In [52]:
info

[{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto',
  'author': 'Raul fernando',
  'previousOrders': '2',
  'previousReviews': '0',
  'date': '21 sep, 2019',
  'worstRating': '1',
  'ratingValue': '3.7',
  'bestRating': '5'},
 {'text': 'El pollo más rico que tiene colombia ❤?❤?',
  'author': 'Andres  ramos',
  'previousOrders': '25',
  'previousReviews': '0',
  'date': '25 jul, 2019',
  'worstRating': '1',
  'ratingValue': '5.0',
  'bestRating': '5'},
 {'text': 'redimi y cupón de $7.000 y me cobraron la cuenta completa sin el descuento del cupon',
  'author': 'Camilo espitia',
  'previousOrders': '266',
  'previousReviews': '0',
  'date': '20 jun, 2019',
  'worstRating': '1',
  'ratingValue': '4.0',
  'bestRating': '5'},
 {'text': 'Excelente servicio, el domiciliario es muy respetuoso y amable',
  'author': 'Laura sanchez',
  'previousOrders': '274',
  'previousReviews': '0',
  'date': '09 jun, 2019',
  'worstRating': '1',
  'ratingValue': '5.0',
  'bestRating': '5'

In [ ]:
import json

with open("ppc_suba_comentarios.json", "w") as f:
    json.dump(info, f)

In [24]:
import json

with open("ppc_suba_comentarios.json", "r") as f:
    comentarios = json.load(f)

In [26]:
comentarios[0]

{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto',
 'author': 'Raul fernando',
 'previousOrders': '2',
 'previousReviews': '0',
 'date': '21 sep, 2019',
 'worstRating': '1',
 'ratingValue': '3.7',
 'bestRating': '5'}

In [36]:
import nltk

nltk.download("stopwords")

from nltk.corpus import stopwords

def removeStopwords( palabras ):
     return [ word for word in palabras if word\
             not in stopwords.words('spanish') ]
    
removeStopwords("de para cuando hola mi nombre es vladimir".split())

['hola', 'nombre', 'vladimir']

In [37]:
Rauldef remove_punct(text):
    signos = ", . ; : ! ? ¿ ¡".split()
    for s in signos:
        text = text.replace(s, "")
    # Todo a minúsculas
    text = text.lower()
    text = " ".join(removeStopwords(text.split()))
    return text.lower()

In [38]:
for com in comentarios:
    com['texto_nopunct'] = remove_punct(com['text'])

In [39]:
comentarios[0]

{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto',
 'author': 'Raul fernando',
 'previousOrders': '2',
 'previousReviews': '0',
 'date': '21 sep, 2019',
 'worstRating': '1',
 'ratingValue': '3.7',
 'bestRating': '5',
 'texto_nopunct': 'parecía arroz día anterior trasnochado mal gusto'}

In [53]:
!pip install fasttext

     |████████████████████████████████| 61kB 9.5MB/s  eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.1-cp37-cp37m-linux_x86_64.whl size=2484310 sha256=2595de8ab862e39fd61dd1fb1b5a20301e4e901e3e01bc4b02265462eca9d1b7
  Stored in directory: /home/ingenieria/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [54]:
len(comentarios)

1170

In [55]:
with open("archivo.txt", "w") as f:
    for c in comentarios[:10]:
        f.write(c['texto_nopunct'])
        f.write("\n")

In [59]:
import fasttext
modelo = fasttext.train_unsupervised("archivo.txt", dim=10)

In [60]:
modelo.get_sentence_vector("faltan cuatro minutos para la una")

array([-0.00724311, -0.01367409, -0.11141652, -0.15393221,  0.04881471,
        0.11089667,  0.09096009, -0.11503991,  0.23437262, -0.1442636 ],
      dtype=float32)

In [62]:
for com in comentarios[:10]:
    com['vector'] = modelo.get_sentence_vector(com['texto_nopunct'])

In [64]:
comentarios[0]

{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto',
 'author': 'Raul fernando',
 'previousOrders': '2',
 'previousReviews': '0',
 'date': '21 sep, 2019',
 'worstRating': '1',
 'ratingValue': '3.7',
 'bestRating': '5',
 'texto_nopunct': 'parecía arroz día anterior trasnochado mal gusto',
 'vector': array([-0.15319133,  0.06751906,  0.13427433,  0.0425053 , -0.11566053,
         0.10251003, -0.05255435, -0.04850954,  0.03822632, -0.03976798],
       dtype=float32)}